In [1]:
import os
from os import path
from glob import glob

import numpy as np
import pandas as pd
from scipy import signal
import json
import matplotlib.pyplot as plt
from matplotlib import cm

%matplotlib widget

In [2]:
data_dir = '../data/eeg_pilot/mouse497154/pilot2_2020.01.15/recording1/'
eeg_cont_dir = path.join(data_dir, 'continuous/Rhythm_FPGA-111.0/*')
eeg_evnt_dir = path.join(data_dir, 'events/Rhythm_FPGA-111.0/TTL_1/*')
eeg_mesg_dir = path.join(data_dir, 'events/Message_Center-904.0/TEXT_group_1/*')

In [3]:
def plot_ts(ts, ax, **kwargs):
    if 'label' not in kwargs.keys():
        kwargs.update(label='ch%s'%ts.name)
    ts.plot(ax=ax, **kwargs)
    ax.legend(loc=1, fontsize=8)
    return

def plot_psd(ts, ax, **kwargs):
    if 'label' not in kwargs.keys():
        kwargs.update(label='ch%s'%ts.name)
    ax.psd(ts, **kwargs)
    ax.legend(loc=1)
    return

# Data streams from EEG recording

## Metadata
Extracted from ```structure.oebin``` file.

In [4]:
with open(path.join(data_dir, 'structure.oebin'), 'r') as f:
    metadata = json.load(f)
# SAMPLE_RATE = metadata['continuous'][0]['sample_rate']
BIT_VOLTS = {x['recorded_processor_index'] : x['bit_volts'] for x in metadata['continuous'][0]['channels']}
with open(path.join(data_dir, 'sync_messages.txt'), 'r') as f:
    lines = f.readlines()
start_time = lines[1].split(':')[-1][:-1]
START_TIME = int(start_time.split('@')[0])
SAMPLE_RATE = int(start_time.split('@')[1][:-2])
print('Start time: %d\nSample rate: %d Hz\nAll metadata:'%(START_TIME, SAMPLE_RATE))
metadata

Start time: 9400320
Sample rate: 10000 Hz
All metadata:


{'GUI version': '0.4.5',
 'continuous': [{'folder_name': 'Rhythm_FPGA-111.0/',
   'sample_rate': 10000,
   'source_processor_name': 'Rhythm FPGA',
   'source_processor_id': 111,
   'source_processor_sub_idx': 0,
   'recorded_processor': 'Rhythm FPGA',
   'recorded_processor_id': 111,
   'num_channels': 32,
   'channels': [{'channel_name': 'CH1',
     'description': 'Headstage data channel',
     'identifier': 'genericdata.continuous',
     'history': 'Rhythm FPGA',
     'bit_volts': 0.19499999284744263,
     'units': 'uV',
     'source_processor_index': 0,
     'recorded_processor_index': 0},
    {'channel_name': 'CH2',
     'description': 'Headstage data channel',
     'identifier': 'genericdata.continuous',
     'history': 'Rhythm FPGA',
     'bit_volts': 0.19499999284744263,
     'units': 'uV',
     'source_processor_index': 1,
     'recorded_processor_index': 1},
    {'channel_name': 'CH3',
     'description': 'Headstage data channel',
     'identifier': 'genericdata.continuous',
 

In [5]:
print('Continuous datasets:')
print([path.basename(x) for x in glob(eeg_cont_dir)])
print('Events datasets:')
print([path.basename(x) for x in glob(eeg_evnt_dir)])
print('Message datasets:')
print([path.basename(x) for x in glob(eeg_mesg_dir)])

Continuous datasets:
['continuous.dat', 'timestamps.npy']
Events datasets:
['timestamps.npy', 'channel_states.npy', 'full_words.npy', 'channels.npy']
Message datasets:
['timestamps.npy', 'text.npy', 'channels.npy']


## Message dataset

In [6]:
dataset = eeg_mesg_dir
messages = pd.DataFrame({
    x.replace('.npy', '') : np.load(path.join(dataset[:-1], x))\
    for x in [path.basename(x) for x in glob(dataset)]
})
messages.timestamps = (messages.timestamps - START_TIME) / SAMPLE_RATE
messages.set_index('timestamps', inplace=True)
messages.text = messages.text.map(lambda x: x.decode('ASCII'))
messages

,text,channels
timestamps,,
727.5008,isoo on T 5%,1
931.2768,iso on 2%,1
1689.3440,iso oFF,1


## Events dataset

In [7]:
dataset = eeg_evnt_dir
events = pd.DataFrame({
    x.replace('.npy', '') : np.load(path.join(dataset[:-1], x))\
    for x in [path.basename(x) for x in glob(dataset)]
})
events.timestamps = (events.timestamps - START_TIME) / SAMPLE_RATE
events.set_index('timestamps', inplace=True)
events

,channel_states,full_words,channels
timestamps,,,
9.8399,8,128,8
9.8599,-8,0,8
9.8799,8,128,8
9.9089,-8,0,8
9.9958,8,128,8
...,...,...,...
3444.1338,-8,0,8
3444.1918,8,128,8
3444.2787,-8,0,8


### Timestamps for synchronization
To be used for synchronization. See https://github.com/open-ephys/sync-barcodes/blob/master/barcodes.py.

In [8]:
f, ax = plt.subplots(1, 1, figsize=(14, 2), tight_layout=True)
_ = events.apply(plot_ts, args=(ax,))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Continuous dataset

In [9]:
dataset = eeg_cont_dir
timestamps = (np.load(path.join(dataset[:-1], 'timestamps.npy')) - START_TIME) / SAMPLE_RATE
data = np.fromfile(path.join(dataset[:-1], 'continuous.dat'), dtype='<i2')*0.195
n_t = int(timestamps.shape[0])
n_c = int(data.shape[0]/timestamps.shape[0])
data = np.reshape(data, (n_t, n_c))
data = pd.DataFrame(
    data=data,
    columns=range(1, 33),
    index=timestamps
#     index=pd.TimedeltaIndex(timestamps, unit='s')
)
data

,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
0.0000,-63.570,-43.875,-61.035,-35.295,-39.390,-25.545,-45.435,-29.835,-631.995,-1.755,...,273.000,1261.260,-46.410,-31.980,-45.825,-46.995,-70.590,-69.420,-66.690,-77.610
0.0001,-61.815,-44.460,-54.015,-23.400,-38.610,-23.595,-43.485,-27.885,-522.600,1.950,...,405.600,1405.950,-38.220,-38.025,-47.580,-41.925,-68.445,-66.300,-67.860,-72.540
0.0002,-57.135,-31.200,-50.310,-19.890,-35.685,-26.715,-46.215,-25.350,-402.870,-1.755,...,510.120,1528.605,-39.975,-30.225,-46.020,-35.100,-67.080,-63.180,-67.470,-76.635
0.0003,-47.190,-31.980,-47.970,-15.405,-28.470,-14.235,-42.510,-27.300,-300.105,-0.585,...,609.960,1637.805,-34.320,-27.885,-47.190,-30.225,-61.035,-56.940,-55.185,-66.300
0.0004,-47.385,-34.320,-37.830,-7.605,-26.910,-11.895,-36.270,-22.620,-185.250,7.020,...,724.230,1776.840,-27.885,-22.815,-29.250,-26.325,-48.165,-47.580,-39.390,-49.725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471.7435,126.750,110.175,64.935,61.425,59.475,81.705,66.885,64.350,-2615.730,55.575,...,-2392.455,-3503.760,54.600,42.510,55.380,49.335,74.490,76.635,116.805,116.025
3471.7436,109.200,93.210,54.600,46.020,48.165,62.790,57.525,60.255,-2833.155,63.180,...,-2615.340,-3732.300,46.800,34.515,50.310,39.975,66.495,68.835,101.985,112.515
3471.7437,96.525,78.780,44.070,24.180,38.805,47.190,48.945,47.775,-3044.535,63.765,...,-3037.905,-4218.435,30.615,25.740,37.830,32.955,49.335,56.355,64.545,87.750
3471.7438,87.165,65.715,46.410,31.005,41.925,45.045,51.285,45.630,-3761.940,54.990,...,-4068.285,-5346.315,38.610,36.075,38.805,34.320,49.335,68.835,63.375,100.035


# Processing EEG signals
[This](http://ims.mf.uni-lj.si/archive/15(1)/21.pdf) might be of interest, and maybe also some papers mentioned in [this](https://www.researchgate.net/post/High_frequency_component_of_EEG_signal_noise_or_information) thread.  

## Visualize raw signals
* The y-axis is on $\mu$V scale
* Ch 1 and 2 are two regular electrodes with proper connections.
* Ch 9 is disconnected. **What is the (5mV) signal there?** This signal is huge. Why doesn't it appear so large in other channels?
* **What are the big spikes (~2mV) in ch 1 and 2?** Head-banging artefacts! Most likely, but not explicitly tested. They correlate well with artefacts in disconnected channels as well, so they are very likely due to straining against the head-plate.

In [10]:
indices = [1, 2, 9]
f, axes = plt.subplots(len(indices), 1, squeeze=False, sharex=True, figsize=(14, 1.5*len(indices)), tight_layout=True)
[data[[indices[i]]].apply(plot_ts, args=(axes[i, 0],), xlim=(226, 227)) for i in range(len(indices))];

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Power spectra of raw signals

In [11]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, sharex=True, figsize=(8, 3.5), tight_layout=True)
[data[[1]].apply(plot_psd, args=(ax1,), Fs=SAMPLE_RATE, NFFT=8192, detrend=i, label=str(i)) for i in [None, 'mean', 'linear']]
ax1.set_title('Compare detrending')
data[[1, 2, 9]].apply(plot_psd, args=(ax2,), Fs=SAMPLE_RATE, NFFT=8192, detrend='linear')
ax2.set_title('Compare channels')
for ax in (ax1, ax2):
    ax.set_xlim(0, 1000)
    ax.set_ylim(0, 50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Low pass filter and remove 60 Hz
For now, I have tried setting the low pass cutoff between 125 and 250 Hz. 125 might be too close to the 30-100 Hz gamma band, but it is the only integer factor of 10,000 below 200, which is why I use it below.

In [12]:
lp_freq = SAMPLE_RATE/80 # assumes sample_rate is 10000, so that cutoff is 125 Hz
lp_b, lp_a = signal.butter(4, lp_freq, 'low', fs=SAMPLE_RATE)
def apply_lp(ts):
    return signal.filtfilt(lp_b, lp_a, ts)
lp_signals = data[[1, 2, 9]].apply(apply_lp)

In [13]:
notch_freq = [60, 120, 180, 240]
width = 4 # filter width in Hz
def apply_notch(ts, freq):
    b, a = signal.iirnotch(freq, Q=60/width, fs=SAMPLE_RATE)
    return signal.filtfilt(b, a, ts)
f_signals = lp_signals.copy()
for freq in notch_freq:
    f_signals = f_signals.apply(apply_notch, args=(freq,))

In [14]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, sharex=True, figsize=(8, 3.5), tight_layout=True)
_ = lp_signals.apply(plot_psd, args=(ax1,), Fs=SAMPLE_RATE, NFFT=8192, detrend='linear')
_ = data[[1, 2, 9]].apply(plot_psd, args=(ax1,), Fs=SAMPLE_RATE, NFFT=8192, detrend='linear', lw=0.2, label=None)
_ = f_signals.apply(plot_psd, args=(ax2,), Fs=SAMPLE_RATE, NFFT=8192, detrend='linear')
_ = data[[1, 2, 9]].apply(plot_psd, args=(ax2,), Fs=SAMPLE_RATE, NFFT=8192, detrend='linear', lw=0.2, label=None)
for ax in (ax1, ax2):
    ax.set_xlim(0, 400)
    ax.set_ylim(0, 50)
ax1.set_title('Low pass @{} Hz'.format(lp_freq))
ax2.set_title('After removing line noise');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Example signals after low-pass filtering

In [15]:
indices = [1, 2, 9]
f, axes = plt.subplots(len(indices), 1, squeeze=False, sharex=True, figsize=(14, 1.5*len(indices)), tight_layout=True)
[data[[indices[i]]].loc[data.index[::20]].apply(plot_ts, args=(axes[i, 0],), ls='--', lw=0.5, xlim=(226, 226.5)) for i in range(len(indices))]
[lp_signals[[indices[i]]].loc[data.index[::20]].apply(plot_ts, args=(axes[i, 0],), xlim=(226, 226.5), label='lp') for i in range(len(indices))];
axes[0, 0].set_ylim(-600, 600)
axes[1, 0].set_ylim(-600, 600);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Signal post low-pass and line noise removal

In [16]:
indices = [1, 2, 9]
f, axes = plt.subplots(len(indices), 1, squeeze=False, sharex=True, figsize=(14, 1.5*len(indices)), tight_layout=True)
[data[[indices[i]]].loc[data.index[::20]].apply(plot_ts, args=(axes[i, 0],), ls='--', lw=0.5, xlim=(226, 226.5)) for i in range(len(indices))]
[f_signals[[indices[i]]].loc[data.index[::20]].apply(plot_ts, args=(axes[i, 0],), xlim=(226, 226.5), label='fully filtered') for i in range(len(indices))];
axes[0, 0].set_ylim(-600, 600)
axes[1, 0].set_ylim(-600, 600);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Notch filters remove too much of signal
The figure below shows the difference in low-pass and low-pass + notch filtered timeseries for a channel. Ideally the difference should simply be a sinusoid of 60 Hz frequency and constant amplitude, corresponding to the nearly constant amplitude line noise.  

However, as can be seen, the difference has low-frequency modulations. This could be because there is a real 60 Hz component to the EEG that is slowly changing. Or the line noise amplitude changes slowly. The latter is quite common, and it is difficult to differentiate the two.

In [17]:
diff_signal = lp_signals-f_signals
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3), gridspec_kw={'width_ratios':[5, 1]}, tight_layout=True)
diff_signal[[1]].loc[data.index[::20]].apply(plot_ts, args=(ax1,))
ax1.set_xlabel('Time (s)')
diff_signal.apply(plot_psd, args=(ax2,), Fs=SAMPLE_RATE, NFFT=8192, detrend='linear')
ax2.set_xlim(-10, 100)
ax2.set_ylim(-70, 70);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Remove line noise without completely filtering 60 Hz

In [18]:
# env = diff_signal.abs().rolling(int(0.05*SAMPLE_RATE), center=True).max()

# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3), gridspec_kw={'width_ratios':[5, 1]}, tight_layout=True)
# diff_signal[[1]].loc[data.index[::20]].apply(plot_ts, args=(ax1,))
# env[[1]].loc[data.index[::20]].apply(plot_ts, args=(ax1,))
# (env[[9]]/env[9].mean()*env[1].mean()).loc[data.index[::20]].apply(plot_ts, args=(ax1,))
# # (env[[1]]*-1).loc[data.index[::20]].apply(plot_ts, args=(ax1,))
# ax1.set_xlabel('Time (s)');

In [19]:
# corr_diff = diff_signal / env * env.mean()
# corr_signal = lp_signals - corr_diff

# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3), gridspec_kw={'width_ratios':[5, 1]}, tight_layout=True)
# lp_signals[[1]].loc[data.index[::20]].apply(plot_ts, args=(ax1,))
# corr_signal[[1]].loc[data.index[::20]].apply(plot_ts, args=(ax1,))
# ax1.set_xlabel('Time (s)')
# corr_signal.dropna(axis=0).apply(plot_psd, args=(ax2,), Fs=SAMPLE_RATE, NFFT=8192, detrend='linear')
# ax2.set_xlim(-10, 100)
# ax2.set_ylim(-70, 70);

In [20]:
# t_anesthesia = (1000, 1600) # get this from iso 2% event

### Use correlation between disconnected and connected channel during anesthesia
The idea was to see how connected channel signal during the down states relates to the disconnected channel (both should be dominated by line noise at this point). Then use this relationship to subtract the line component from the connected channels.  
This method does not work (see below). Presumably the relationship between line noise in connected and disconnected channels changes over time.

In [21]:
# f, (ax, ax2) = plt.subplots(1, 2, figsize=(8, 3), tight_layout=True)
# lp_signals.iloc[t_anesthesia[0]*SAMPLE_RATE:t_anesthesia[1]*SAMPLE_RATE:20][1].plot(ax=ax2)
# lp_signals.iloc[t_anesthesia[0]*SAMPLE_RATE:t_anesthesia[1]*SAMPLE_RATE:20][9].apply(lambda x: x/100).plot(ax=ax2)
# ax2.legend(loc=1, fontsize=8)

# _, _, _, im = ax.hist2d(
#     lp_signals.iloc[t_anesthesia[0]*SAMPLE_RATE:t_anesthesia[1]*SAMPLE_RATE:20][1],
#     lp_signals.iloc[t_anesthesia[0]*SAMPLE_RATE:t_anesthesia[1]*SAMPLE_RATE:20][9],
#     bins=(100, 100), range=[[-50, 50], [-6000, 6000]], cmin=10
# )
# f.colorbar(im, ax=ax)
# ax.set_xlabel('Channel 1')
# ax.set_ylabel('(Channel 9) / 100')
# ax.annotate('Correlation during anesthesia', (0.01, 0.99), xycoords='axes fraction', va='top');

In [22]:
# df = data[[1, 2, 9]].iloc[t_anesthesia[0]*SAMPLE_RATE:t_anesthesia[1]*SAMPLE_RATE:20]
# anesthesia_corr = df.corr() * (df.std().values[:, np.newaxis]/df.std().values)
# df = lp_signals.iloc[0:t_anesthesia[0]*SAMPLE_RATE:20].corr()
# pre_corr = df.corr() * df.corr() * (df.std().values[:, np.newaxis]/df.std().values)
# df = lp_signals.iloc[t_anesthesia[1]*SAMPLE_RATE::20].corr()
# post_corr = df.corr() * df.corr() * (df.std().values[:, np.newaxis]/df.std().values)
# pd.concat([anesthesia_corr, pre_corr, post_corr], axis=1)

In [23]:
# df = lp_signals - pd.DataFrame(lp_signals[9]).dot(pd.DataFrame(anesthesia_corr[9]).T)
# df.shape

In [24]:
# indices = [0, 1, 8]
# f, axes = plt.subplots(len(indices), 1, squeeze=False, sharex=True, figsize=(14, 2*len(indices)), tight_layout=True)
# for i, ax in enumerate(axes.T[0]):
#     data[indices[i]+1].loc[data.index[::20]].plot(ax=ax, ls='--', lw=0.5)
#     df[indices[i]+1].loc[data.index[::20]].plot(ax=ax, label='CH%d'%(indices[i]+1))
#     f_signals[indices[i]+1].loc[data.index[::20]].plot(ax=ax, label='CH%d'%(indices[i]+1))
#     ax.legend(loc=1)
# #     ax.set_xlim(226, 226.5)
# #     if indices[i]+1 != 9:
# #         ax.set_ylim(-600, 600)

In [25]:
# f, ax = plt.subplots(1, 1, figsize=(4, 3), tight_layout=True)
# [ax.psd(df[i], Fs=SAMPLE_RATE, NFFT=8192, detrend='linear', label='ch%d'%i) for i in [1, 2]]
# [ax.psd(data[i], Fs=SAMPLE_RATE, NFFT=8192, detrend='linear', lw=0.2) for i in [1, 2]]
# ax.set_xlim(0, 125)
# ax.set_ylim(0, 40)
# ax.legend(loc=1)

### How else can we remove only the line-noise component of 60 Hz without removing any intrinsic 60 Hz signals?

In [26]:
# interesting time intervals for quick testing before looking at all of data
toi_up_down = (1269, 1270) # transition from up to down state during anesthesia
toi_head_large = (272.4, 273.4) # transition from large head-banging artefact to normal-looking eeg
toi_head_small = (259, 260) # transition from small head-banging artefact to normal-looking eeg

## Remove head-banging artefacts
In the signals above, it looks like when the mouse is straining against the headplate, the amplitude of original signal is much larger than amplitude of filtered signal. Below, I simply run a short window, compute the mean absolute difference between original and filtered signal and plot it on top of the original signals, to see how well this huristic method works.  

The image below shows the window-averaged absolute difference (dark red trace, right y-axis) during a weak and strong head-banging event. It can be used to set thresholds to eliminate such events.

In [18]:
convolution_window = int(0.5 * SAMPLE_RATE) # average the difference over 0.5 second window
def get_absdelta(ts):
    absd = np.convolve(
        np.abs(ts - f_signals[ts.name]),
        np.ones(convolution_window), mode='same',
    ) / convolution_window
    return (absd - absd.mean()) / absd.std()
absdelta = data[[1, 2, 9]].apply(get_absdelta)

In [19]:
indices = [1, 2, 9]
f, axes = plt.subplots(len(indices), 1, squeeze=False, sharex=True, figsize=(14, 1.5*len(indices)), tight_layout=True)
for i, ax in enumerate(axes.T[0]):
    data[[indices[i]]].loc[data.index[::20]].apply(plot_ts, args=(ax,), ls='--', lw=0.5)
    f_signals[[indices[i]]].loc[data.index[::20]].apply(plot_ts, args=(ax,))
    ax2 = ax.twinx()
    absdelta.abs()[[indices[i]]].loc[data.index[::20]].apply(plot_ts, args=(ax2,), c=cm.Reds(0.9, 0.99))
axes[0, 0].set_ylim(-300, 300)
axes[1, 0].set_ylim(-300, 300);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
absdelta['fp'] = absdelta.apply(lambda x: np.abs(x)).mean(axis=1) < 0.5
absdelta['fp'] = np.convolve(absdelta.fp.astype('float'), np.ones(5000)/5000, mode='same')
absdelta['fp'] = absdelta.fp > 0.5

In [21]:
f, ax = plt.subplots(1, 1, figsize=(12, 1.5), tight_layout=True)
absdelta[['fp']].astype(float).loc[data.index[::20]].apply(plot_ts, args=(ax,), c=cm.Reds(0.9, 0.99))
ax2 = ax.twinx()
pd.DataFrame(f_signals[9]*absdelta.fp).loc[data.index[::20]].apply(plot_ts, args=(ax2,))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0    None
dtype: object

# Plot spectrograms

In [27]:
f, ax = plt.subplots(1, 1, figsize=(12, 3), tight_layout=True)
sp, _, _, im = ax.specgram(data.loc[data.index[::20]][1], Fs=SAMPLE_RATE/20, NFFT=256, detrend='linear')
f.colorbar(im, ax=ax)
# ax.set_ylim(0, 140)
im.set_clim(-40, 40);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
f, ax = plt.subplots(1, 1, figsize=(12, 3), tight_layout=True)
sp, _, _, im = ax.specgram(f_signals.loc[data.index[::1]][1], Fs=SAMPLE_RATE/1, NFFT=256*32, detrend='linear')
f.colorbar(im, ax=ax)
ax.set_ylim(0, 200)
im.set_clim(-40, 40);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …